In [5]:
# Common ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Assuming grayscale images and input size 224x224
target_size = (224, 224)
batch_size = 32

# Load test sets
dti_test_gen = datagen.flow_from_directory(
    dti_test_path, target_size=target_size, color_mode='grayscale', class_mode='categorical', batch_size=batch_size, shuffle=False)

mri_test_gen = datagen.flow_from_directory(
    mri_test_path, target_size=target_size, color_mode='grayscale', class_mode='categorical', batch_size=batch_size, shuffle=False)

pet_test_gen = datagen.flow_from_directory(
    pet_test_path, target_size=target_size, color_mode='grayscale', class_mode='categorical', batch_size=batch_size, shuffle=False)


Found 10443 images belonging to 3 classes.
Found 2880 images belonging to 3 classes.
Found 85827 images belonging to 3 classes.


In [ ]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from collections import Counter

# --- Class Setup ---
classes = ['AD', 'CN', 'MCI']
class_indices = {name: idx for idx, name in enumerate(classes)}

# --- Load Models ---
mri_model = load_model("D:/CIP_MRI/MRI_Model.h5")
pet_model = load_model("D:/CIP_PET/PET_Model.h5")
dti_model = load_model("D:/CIP/DTI_Model.h5")

# --- Preprocessing Functions ---
def preprocess(img_path, normalize=True):
    img = image.load_img(img_path, color_mode='grayscale', target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = img_array / 127.5 - 1 if not normalize else img_array / 255.0
    return np.expand_dims(img_array, axis=0)

# --- Dataset Paths ---
mri_test_path = "D:/CIP_MRI/split/test"
pet_test_path = "D:/CIP_PET/split_pet_images/test"
dti_test_path = "D:/CIP/split_dti_images/test"

# --- Get All Fused Predictions ---
correct = 0
total = 0

for label in classes:
    mri_dir = os.path.join(mri_test_path, label)
    pet_dir = os.path.join(pet_test_path, label)
    dti_dir = os.path.join(dti_test_path, label)

    mri_files = [f for f in os.listdir(mri_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    
    for file in mri_files:
        mri_img_path = os.path.join(mri_dir, file)
        pet_img_path = os.path.join(pet_dir, file) if os.path.exists(os.path.join(pet_dir, file)) else None
        dti_img_path = os.path.join(dti_dir, file) if os.path.exists(os.path.join(dti_dir, file)) else None
        
        if not (pet_img_path and dti_img_path):  # skip if any missing
            continue
        
        # Preprocess
        mri_img = preprocess(mri_img_path, normalize=False)
        pet_img = preprocess(pet_img_path)
        dti_img = preprocess(dti_img_path)
        
        # Predict
        pred_mri = mri_model.predict(mri_img, verbose=0)
        pred_pet = pet_model.predict(pet_img, verbose=0)
        pred_dti = dti_model.predict(dti_img, verbose=0)
        
        # Fusion (majority voting)
        preds = [np.argmax(pred_mri), np.argmax(pred_pet), np.argmax(pred_dti)]
        if len(set(preds)) == 3:
            # If tie, use highest confidence
            confidences = [np.max(pred_mri), np.max(pred_pet), np.max(pred_dti)]
            final_pred = np.argmax([pred_mri, pred_pet, pred_dti][np.argmax(confidences)])
        else:
            final_pred = Counter(preds).most_common(1)[0][0]
        
        true_label = class_indices[label]
        
        if final_pred == true_label:
            correct += 1
        total += 1

# --- Final Accuracy ---
fused_accuracy = correct / total
print(f"\n✅ Fused Model Accuracy: {fused_accuracy:.4f} ({correct}/{total})")


In [6]:
        import os
        import random
        import numpy as np
        from tensorflow.keras.models import load_model
        from tensorflow.keras.preprocessing import image
        
        # Class Names
        classes = ['AD', 'CN', 'MCI']
        
        # Load Models
        mri_model = load_model("D:/CIP/final_mri_model.h5")
        pet_model = load_model("D:/CIP_PET/PET_Model.h5")
        dti_model = load_model("D:/CIP/DTI_Model.h5")
        
        # Preprocessing Functions 
        def preprocess_mri(img_path):
            img = image.load_img(img_path, color_mode='grayscale', target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = img_array / 255.0  # [0, 1]
        
            return np.expand_dims(img_array, axis=0)
        
        def preprocess_pet_dti(img_path):
            img = image.load_img(img_path, color_mode='grayscale', target_size=(224, 224))
            img_array = image.img_to_array(img)
            img_array = img_array / 255.0  # Normalize to [0, 1]
            return np.expand_dims(img_array, axis=0)
        
        #Function to Get Random Images from the Same Category 
        def get_random_category_image_paths(mri_path, pet_path, dti_path):
            categories = os.listdir(mri_path)
            categories = [cat for cat in categories if os.path.isdir(os.path.join(mri_path, cat))]
            chosen_category = random.choice(categories)
        
            def pick_random_image(category_folder):
                category_path = os.path.join(category_folder, chosen_category)
                files = [f for f in os.listdir(category_path) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
                if not files:
                    raise ValueError(f"No images found in {category_path}")
                return os.path.join(category_path, random.choice(files))
        
            mri_img = pick_random_image(mri_path)
            pet_img = pick_random_image(pet_path)
            dti_img = pick_random_image(dti_path)
            
            return chosen_category, mri_img, pet_img, dti_img
        
        #  Paths to Test Sets 
        mri_test_path = "D:/CIP_MRI/split/test"
        pet_test_path = "D:/CIP_PET/split_pet_images/test"
        dti_test_path = "D:/CIP/split_dti_images/test"
        
        # Load Random Images from Same Category 
        true_category, mri_img_path, pet_img_path, dti_img_path = get_random_category_image_paths(
            mri_test_path, pet_test_path, dti_test_path
        )
        
        print(f"\n🗂 Selected Ground Truth Category: {true_category}")
        print(" DTI Image:", dti_img_path)
        
        #Preprocess & Predict
        mri_img = preprocess_mri(mri_img_path)
        pet_img = preprocess_pet_dti(pet_img_path)
        dti_img = preprocess_pet_dti(dti_img_path)
        
        pred_mri = mri_model.predict(mri_img)
        pred_pet = pet_model.predict(pet_img)
        pred_dti = dti_model.predict(dti_img)
        
        #  Combine Predictions
        preds = [np.argmax(pred_mri), np.argmax(pred_pet), np.argmax(pred_dti)]
        
        if len(set(preds)) == 3:  # All different (tie)
            prob_mri = np.max(pred_mri)
            prob_pet = np.max(pred_pet)
            prob_dti = np.max(pred_dti)
            confidence_preds = [prob_mri, prob_pet, prob_dti]
            final_class = classes[np.argmax(confidence_preds)]
            print("\n✅ Final Fused Diagnosis (Max Confidence):", final_class)
            print(" MRI Confidence:", prob_mri)
            print(" PET Confidence:", prob_pet)
            print(" DTI Confidence:", prob_dti)
        else:
            final_vote = max(set(preds), key=preds.count)
            print("\n✅ Final Fused Diagnosis :", classes[final_vote])
        
        
        
        # --- Printing Class Probabilities for Each Model ---
        print("\n Predictions with Probabilities:")
        print("MRI Prediction:", classes[np.argmax(pred_mri)], "| Probabilities:",pred_mri[0])
        print("PET Prediction:", classes[np.argmax(pred_pet)], "| Probabilities:", pred_pet[0])
        print("DTI Prediction:", classes[np.argmax(pred_dti)], "| Probabilities:",pred_dti[0])


🗂 Selected Ground Truth Category: CN
 DTI Image: D:/CIP/split_dti_images/test\CN\ADNI_021_S_4421_MR_Fractional_Aniso.__br_raw_20120719155740963_39_S158110_I316999.png
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 219ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 233ms/step

✅ Final Fused Diagnosis : CN

 Predictions with Probabilities:
MRI Prediction: CN | Probabilities: [2.7764627e-18 1.0000000e+00 6.9377040e-13]
PET Prediction: CN | Probabilities: [0. 1. 0.]
DTI Prediction: CN | Probabilities: [1.7981088e-07 5.9882462e-01 4.0117529e-01]


In [5]:
import os
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tqdm import tqdm  # for progress bar

# Class Names
classes = ['AD', 'CN', 'MCI']

# Load Models
mri_model = load_model("D:/CIP/final_mri_model.h5")
pet_model = load_model("D:/CIP_PET/PET_Model.h5")
dti_model = load_model("D:/CIP/DTI_Model.h5")

# Preprocessing Functions
def preprocess_img(img_path):
    img = image.load_img(img_path, color_mode='grayscale', target_size=(224, 224))
    img_array = image.img_to_array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

# Paths to Test Sets
mri_test_path = "D:/CIP_MRI/split/test"
pet_test_path = "D:/CIP_PET/split_pet_images/test"
dti_test_path = "D:/CIP/split_dti_images/test"

# Initialize counters
total = 0
correct = 0

# Loop through each class folder
for category in os.listdir(mri_test_path):
    mri_cat_path = os.path.join(mri_test_path, category)
    pet_cat_path = os.path.join(pet_test_path, category)
    dti_cat_path = os.path.join(dti_test_path, category)

    if not (os.path.isdir(mri_cat_path) and os.path.isdir(pet_cat_path) and os.path.isdir(dti_cat_path)):
        continue

    mri_images = sorted([f for f in os.listdir(mri_cat_path) if f.lower().endswith(('.jpg', '.png'))])
    pet_images = sorted([f for f in os.listdir(pet_cat_path) if f.lower().endswith(('.jpg', '.png'))])
    dti_images = sorted([f for f in os.listdir(dti_cat_path) if f.lower().endswith(('.jpg', '.png'))])

    # Make sure the same number of images are present in all three modalities
    min_len = min(len(mri_images), len(pet_images), len(dti_images))
    for i in tqdm(range(min_len), desc=f"Processing {category}"):
        mri_img = preprocess_img(os.path.join(mri_cat_path, mri_images[i]))
        pet_img = preprocess_img(os.path.join(pet_cat_path, pet_images[i]))
        dti_img = preprocess_img(os.path.join(dti_cat_path, dti_images[i]))

        pred_mri = mri_model.predict(mri_img, verbose=0)
        pred_pet = pet_model.predict(pet_img, verbose=0)
        pred_dti = dti_model.predict(dti_img, verbose=0)

        preds = [np.argmax(pred_mri), np.argmax(pred_pet), np.argmax(pred_dti)]

        if len(set(preds)) == 3:
            # Tie - pick highest confidence
            confidences = [np.max(pred_mri), np.max(pred_pet), np.max(pred_dti)]
            final_class = np.argmax(confidences)
        else:
            # Majority vote
            final_class = max(set(preds), key=preds.count)

        if classes[final_class] == category:
            correct += 1
        total += 1

# ✅ Print Fused Accuracy
accuracy = (correct / total) * 100
print(f"\n✅ Final Fused Accuracy: {accuracy:.2f}% ({correct}/{total})")


Processing MCI: 100%|██████████| 960/960 [10:05<00:00,  1.58it/s]


✅ Final Fused Accuracy: 98.09% (2825/2880)
